# Análisis de datos utilizando PyRoot


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import ROOT
ROOT.gROOT.Reset()

In [3]:
ROOT.gSystem.AddDynamicPath("/scratch/cms/MadGraph/MG5_aMC_v2_5_2/Delphes/")

In [4]:
ROOT.gSystem.Load("libDelphes");

TClassTable::Add:0: RuntimeWarning: class TBuffer already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TDirectory already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TRootIOCtor already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TBrowser already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TApplicationImp already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TApplication already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TAtt3D already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TAttAxis already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TAttBBox already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TAttFill already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TAttLine already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TAttMarker already in TClassTable
TClassTable::Add:0: RuntimeWarning: class TAttPad already in TClassTable
TClassTable::Add:0: Runt

SystemError: problem in C++; program state has been reset

In [4]:
import os

In [5]:
cwd = os.getcwd()
print cwd

/scratch/cms/Analisis_PyRoot


In [6]:
import commands

In [7]:
tmp = commands.getstatusoutput('ls ../Datos/Signal_2j_mu-nu-Run0/Events')

In [8]:
tmp[1]

'run_01\nrun_02\nrun_03\nrun_04\nrun_05\nrun_06\nrun_07\nrun_08\nrun_09\nrun_10'

In [9]:
Runs = tmp[1].split("\n") 

In [10]:
Runs

['run_01',
 'run_02',
 'run_03',
 'run_04',
 'run_05',
 'run_06',
 'run_07',
 'run_08',
 'run_09',
 'run_10']

In [11]:
MainChain=ROOT.TChain("Delphes")

In [12]:
for run in range(len(Runs)):
    imput = "../Datos/Signal_2j_mu-nu-Run0/Events/"+Runs[run]+"/tag_1_delphes_events.root"
    MainChain.Add(imput)
    

1

1

1

1

1

1

1

1

1

1

In [15]:
MainChain.Print()

In [16]:
MainChain.Show()

In [17]:
MainChain.GetEntry(1)

165774

In [18]:
t=ROOT.TBrowser()

In [19]:
numberOfEntries = MainChain.GetEntries()

In [20]:
numberOfEntries

355148L

In [21]:
branchJet = MainChain.GetBranch("Jet")

In [22]:
branchJet.


SyntaxError: invalid syntax (<ipython-input-22-df0087eaa13f>, line 1)

In [58]:
pt=branchJet.GetLeaf("camilo")

In [60]:
pt.GetValueLongDouble

<ROOT.TLeaf object at 0x(nil)>